In [4]:
#Install
%pip install matplotlib
%pip install pandas
%pip install mne
%pip install PyQt6 
%pip install pyqtgraph

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import matplotlib
import pandas
import mne
import PyQt6 # For pyqtgraph
import pyqtgraph

In [11]:
!pip install brainflow
import brainflow

In [14]:
import argparse
import time

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

User API
BrainFlow User API has three main modules:

- BoardShim to read data from a board, it calls methods from underlying BoardController library
- DataFilter to perform signal processing, it calls methods from underlying DataHandler library
- MLModel to calculate derivative metrics, it calls methods from underlying MLModule library

In [ ]:
# Step 1: Use BoardShim to read data from a board

# To simulate "command-line arguments"
import sys

# just the required arguments for now.
sys.argv = ['realtime.ipynb',  
			'--board-id', '0',
			'--serial-port', 'COM9',
			'--mac-address', '[mac-address]'] 

# initialize objects
BoardShim.enable_dev_board_logger()
parser = argparse.ArgumentParser()

# set arguments
# use docs to check which parameters are required for specific board, e.g. for Cyton - set serial port
parser.add_argument('--timeout', type=int, help='timeout for device discovery or connection', required=False, default=0) # 18 for timeout error	
parser.add_argument('--ip-port', type=int, help='ip port', required=False, default=0)
parser.add_argument('--ip-protocol', type=int, help='ip protocol, check IpProtocolType enum', required=False,
                        default=0)

parser.add_argument('--ip-address', type=str, help='ip address', required=False, default='') # For socket connection (I don't think this applies to us)
parser.add_argument('--serial-port', type=str, help='serial port', required=False, default='') # For socket connection (I don't think this applies to us)

parser.add_argument('--mac-address', type=str, help='mac address', required=False, default='') # This should apply to us; bluetooth based board

parser.add_argument('--other-info', type=str, help='other info', required=False, default='') # Extra

parser.add_argument('--serial-number', type=str, help='serial number', required=False, default='') # From board
parser.add_argument('--board-id', type=int, help='board id, check docs to get a list of supported boards', required=True) # 0 for Cyton board
parser.add_argument('--file', type=str, help='file', required=False, default='') # Output file?

parser.add_argument('--master-board', type=int, help='master board id for streaming and playback boards', required=False, default=BoardIds.NO_BOARD) # Not sure


# parse arguments
args = parser.parse_args(sys.argv[1:]) # Excluding the notebook name

# set arguments
params = BrainFlowInputParams()
params.ip_port = args.ip_port
params.serial_port = args.serial_port
params.mac_address = args.mac_address
params.other_info = args.other_info
params.serial_number = args.serial_number
params.ip_address = args.ip_address
params.ip_protocol = args.ip_protocol
params.timeout = args.timeout
params.file = args.file
params.master_board = args.master_board

# set board
board = BoardShim(args.board_id, params)
board.prepare_session()
board.start_stream ()
time.sleep(10)
# data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer
data = board.get_board_data()  # get all data and remove it from internal buffer
board.stop_stream()
board.release_session()

BrainFlowError: UNABLE_TO_OPEN_PORT_ERROR:2 unable to prepare streaming session